# 数据准备

In [34]:
import os  
import torch  
import pandas as pd  
import matplotlib.pyplot as plt
  
    #step1:加载数据集
# 设置数据集的路径  
data_dir = '../../../DataFiles/ODS/strain'   
# 创建空的列表来存储所有的DataFrame  
X_data0 = []  
y_data0 = []
# 遍历指定目录下的所有文件  
for filename in os.listdir(data_dir):  
    if filename.endswith('.csv'):  
        # 构建文件的完整路径  
        file_path = os.path.join(data_dir, filename)  
        # 读取Excel文件  
        df = pd.read_csv(file_path)
        serie=df.iloc[:, 0]
        version_without_extension = filename.rsplit(".", 1)[0] 
        serie.name=version_without_extension
        # 将读取的DataFrame添加到列表中  
        X_data0.append(serie)
        y_data0.append(version_without_extension)

    #step2:遍历X_data0中的每个DataFrame，并删除非数字数据  
cleaned_data = []  
for df in X_data0:  
    df_cleaned = df[pd.to_numeric(df, errors='coerce').notnull()]  
    cleaned_data.append(df_cleaned)  
 

    #step3:将X_data0中的每个DataFrame的所有元素变为float类型 
for i, df in enumerate(cleaned_data):  
    X_data0[i] = df.astype(float) 

# 初始化X_data1来存储结果  
X_data1 = []  
  
# 遍历X_data0中的每个Series  
for series in X_data0:  
    # 计算步长（由于重叠率为50%，步长为序列长度的一半）  
    step_size = 1024  # 因为2048的一半是1024，且我们想要50%的重叠  
    # 初始化一个空的list来存储子序列  
    subsequences = []  
    # 使用range和步长来生成子序列  
    for i in range(0, len(series) - 2048+ 1, step_size):  
        # 确保切片是一维的，并转换为Series（虽然这里切片已经是Series）  
        subsequence = series.iloc[i:i+2048]  
        subsequences.append(subsequence) 
    X_data1.append(subsequences)  
print((type(X_data1),len(X_data1),len(X_data1[0])))

import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import LabelEncoder  
from keras.utils import to_categorical
import time  
from keras.callbacks import Callback  
import torch 

# 假设 X_data1 已经定义好了  
all_data = []  
all_labels = []  
  
for sublist in X_data1:  
    for series in sublist:  
        # 假设每个series是pandas Series  
        all_data.append(series.values)  
        all_labels.append(series.name)  
  
# 将数据转换为numpy数组  
data_array = np.array(all_data)  
# 将标签转换为编码  

label_encoder = LabelEncoder()  
labels_encoded = label_encoder.fit_transform(all_labels)  
  
# 如果需要one-hot编码  
labels_one_hot = to_categorical(labels_encoded)  
  
# 这里你可能需要reshape data_array 以符合LSTM的输入要求  
# LSTM通常要求输入形状为 (samples, time_steps, features)  
# 假设每个series的长度相同，或者你可以填充/截断它们以具有相同的长度  
# time_steps = data_array.shape[1]  # 假设所有series长度相同  
# data_array = data_array.reshape(-1, time_steps, 1)  # 增加一个维度以匹配LSTM的输入要求

# 分割数据集  
X_train, X_temp, y_train, y_temp = train_test_split(data_array, labels_one_hot, test_size=0.3, random_state=42)  
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print((type(X_train),X_train.shape,type(y_train),y_train.shape))
X_train_first_12 = X_train[:12]  

# Go_EMD的实现和效果展示
- 我的个人理解：[Go_EMD](./Go_EMD.md)；
- Go_EMD是定制化于特定场景的，本代码应用场景是提取出包含主要能量成分的 IMF(目标函数是IMF 的能量);

## step1:设置Go_EMD的函数

In [35]:
import numpy as np
from PyEMD import EMD
import matplotlib.pyplot as plt

def goal_function(imf):
    """
    目标函数：计算IMF的能量（平方和）
    """
    return np.sum(imf ** 2)

def Go_EMD(signal, max_imfs=None, energy_threshold=0.1):
    """
    使用Go-EMD对信号进行分解，提取目标导向的IMFs
    
    参数:
    signal (array-like): 输入信号
    max_imfs (int): 最大提取IMF的数量
    energy_threshold (float): 能量阈值，控制分解的停止条件
    
    返回:
    selected_imfs (array-like): 提取的IMFs
    """
    emd = EMD()
    residue = signal.copy()
    selected_imfs = []
    
    # EMD 分解
    imfs = emd.emd(residue)
    num_imfs = len(imfs)
    for i in range(num_imfs):
        imf = imfs[i]
        energy = goal_function(imf)
        # 如果能量大于阈值，保留该IMF
        if energy > energy_threshold * goal_function(signal):
            selected_imfs.append(imf)
        
        # 更新残差
        residue -= imf
        
        if max_imfs and len(selected_imfs) >= max_imfs:
            break
    
    return np.array(selected_imfs)

## step2:对原始信号进行Go_EMD操作，获得IMF组；

In [36]:
import numpy as np
from PyEMD import CEEMDAN
# 假设你已经有一个信号
signal = X_train_first_12[0]  # 将长度为2048的信号替换为你的信号


# 使用Go_EMD对信号进行分解
IMFs = Go_EMD(signal, max_imfs=4, energy_threshold=0.00001)


# 计算残差
residual = signal - np.sum(IMFs, axis=0)

# 信号重构
reconstructed_signal = np.sum(IMFs, axis=0)

print("IMFs shape:", IMFs.shape)
print("Residual shape:", residual.shape)
print("Reconstructed signal shape:", reconstructed_signal.shape)


## step3:显示IMF组和原始信号和重构信号；
- 每个IMF都代表了原始信号的某种固有振动模式信息；
- IMF包含的信息可通过[筛选](../../ModelsContrast/110spmemdlstmvibration.ipynb)后，以达到去噪的效果；之后再重构；

In [37]:
plt.plot(signal, 'r')
plt.title("原始信号")
plt.show()

plt.figure(figsize=(15,10))
for num in range(4):
    plt.subplot(4+1, 1, num+2)
    plt.plot(IMFs[num])
    plt.title("IMF "+str(num+1))
plt.show()

plt.plot(reconstructed_signal, 'r')
plt.title("重构信号")
plt.show()

plt.plot(residual, 'r')
plt.title("残差信号")
plt.show()
